## Get data from copernicus

In [ ]:
import geopandas as gpd

china = gpd.read_file(
    "/home/ivanra/documents/PyPSA-China-PIK/resources/data/regions/regions_onshore.geojson"
)
bbox_china = china.total_bounds[[2, 3, 0, 1]].tolist()

## 1. Authenticate

In [ ]:
import json
import os
import time

import jwt
import requests

# Load saved key from filesystem
TOKEN_PATH = os.path.expanduser("~/documents/token_land_cover_copernicus.json")
service_key = json.load(open(TOKEN_PATH, "rb"))

private_key = service_key["private_key"].encode("utf-8")

claim_set = {
    "iss": service_key["client_id"],
    "sub": service_key["user_id"],
    "aud": service_key["token_uri"],
    "iat": int(time.time()),
    "exp": int(time.time() + (60 * 60)),
}
grant = jwt.encode(claim_set, private_key, algorithm="RS256")
base_url = claim_set["aud"]
token_request = requests.post(
    base_url,
    headers={
        "Accept": "application/json",
        "Content-Type": "application/x-www-form-urlencoded",
    },
    data=f"grant_type=urn:ietf:params:oauth:grant-type:jwt-bearer&assertion={grant}",
)
if token_request.status_code != 200:
    print(token_request.text)
    exit(1)
token = token_request.json()

In [ ]:
token

In [ ]:
auth_rqst = requests.get(
    "https://land.copernicus.eu/api",
    headers={
        "Authorization": f"Bearer {token['access_token']}",
        "Accept": "application/json",
    },
)
if auth_rqst.status_code != 200:
    print(auth_rqst.text)
    exit(1)
auth_rqst.json()

In [ ]:
auth_rqst = requests.get(
    "https://land.copernicus.eu/api/en/products/global-dynamic-land-cover/copernicus-global-land-service-land-cover-100m-collection-3-epoch-2019-globe",
    headers={
        "Authorization": f"Bearer {token['access_token']}",
        "Accept": "application/json",
    },
)
if auth_rqst.status_code != 200:
    print(auth_rqst.text)
    exit(1)
auth_rqst

In [ ]:
auth_rqst.json()

In [ ]:
def search_items(json_return: dict, target_name="global-dynamic-land-cover"):
    return [
        itm["@id"] for itm in json_return["items"] if itm["@id"].find(target_name) != -1
    ]


def find_data_set_id(name="global-dynamic-land-cover") -> dict:
    """Find the catalogue ID of the dataset

    Args:
        name (str, optional): dataset product name. Defaults to "global-dynamic-land-cover".

    Returns:
        dict: the results
    """
    bid = 0
    # batches of 25
    while True:
        batch = "" if bid == 0 else f"b_start={bid}&"
        search_req = requests.get(
            f"https://land.copernicus.eu/api/@search?{batch}portal_type=DataSet&metadata_fields=UID&metadata_fields=dataset_full_format&&metadata_fields=dataset_download_information",
            headers={"Accept": "application/json"},
        )
        if search_req.status_code != 200:
            print(search_req.text)
            exit(1)
        search_results = search_req.json()
        res = search_items(search_results, target_name=name)
        if res == []:
            bid += 25
        else:
            return search_results


res = find_data_set_id()

In [ ]:
res_list = [r for r in res["items"] if r["@id"].find("2019") != -1]
download_info_id = res_list[0]["dataset_download_information"]["items"][0]["@id"]
download_id = res_list[0]["UID"]
dataset_url = res_list[0]["@id"]
download_info_id, download_id, dataset_url

In [ ]:
dataset_url = "https://land.copernicus.eu/api/en/products/global-dynamic-land-cover/copernicus-global-land-service-land-cover-100m-collection-3-epoch-2019-globe"
headers = {
    "Authorization": f"Bearer {token['access_token']}",
    "Accept": "application/json",
    "Content-type": "application/json",
}

json_data = {
    "Datasets": [
        {
            "DatasetID": download_id,
            "DatasetDownloadInformationID": download_info_id,
            "Layer": "Cover Fraction: Built-up ",
            "NUTS": "CN",
            "OutputFormat": "Geotiff",
            "OutputGCS": "EPSG:4326",
        }
    ]
}

data = requests.post(dataset_url, headers=headers, json=json_data)
data

In [ ]:
data.json().keys()

In [ ]:
status = requests.get(
    "https://land.copernicus.eu/api/@datarequest_search?status=In_progress",
    headers={
        "Accept": "application/json",
        "Authorization": f"Bearer {token['access_token']}",
    },
)
status.json()